# Lab 1
The Set Cover Problem has a complex combinatorial structure with many local minima. Hill Climbing often gets stuck in these minima, whereas Simulated Annealing, with its ability to temporarily accept worse solutions, can escape from these traps and continue exploring other areas of the solution space.

The balance between exploration and exploitation offered by Simulated Annealing is particularly effective for problems like Set Cover, where the number of possible combinations is vast, and there is no straightforward path to continuous improvement.

In conclusion, Simulated Annealing is preferable for solving the Set Cover Problem because:

- It can overcome local minima.
- It explores the solution space more thoroughly.
- It is more resilient when dealing with complex combinatorial problems.

In [16]:
from random import random, seed
from itertools import product
import numpy as np
from matplotlib import pyplot as plt
from icecream import ic
from tqdm import tqdm

## Reproducible Initialization

In [17]:
UNIVERSE_SIZES = [100, 1000, 10_000, 100_000, 100_000, 100_000]
NUM_SETS_LIST = [10, 100, 1000, 10_000, 10_000, 10_000]
DENSITIES = [0.2, 0.2, 0.2, 0.1, 0.2, 0.3]

## Helper Functions

In [18]:
def valid(solution):
    """Checks whether the solution is valid (covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))

def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

def fitness(solution: np.array) -> tuple:
    """Returns a tuple (validity, negative cost) to maximize validity and minimize cost."""
    return (valid(solution), -cost(solution))

## Optimization Function with Simulated Annealing








In [19]:
def simulated_annealing(sets, costs, num_sets, density, rng, initial_temp=2100.0, cooling_rate=0.995, min_temp=1e-3, max_iter=20_000):
    if (num_sets) < 1000:
        current_solution = rng.random(num_sets) < 1
    else:  
        current_solution = rng.random(num_sets) < 0.5
    #Fitness of current solution
    current_fitness = fitness(current_solution)
    best_solution = current_solution
    best_fitness = current_fitness
    temperature = initial_temp
    history = [current_fitness[1]]

    for iteration in tqdm(range(max_iter), desc="Simulated Annealing Progress"):
        new_solution = current_solution.copy()
        #Swapping of one set from the current solution with another one
        swap_idx = np.random.randint(num_sets)
        new_solution[swap_idx] = not new_solution[swap_idx]  
        new_fitness = fitness(new_solution)

        if not valid(new_solution):
            continue
        
        if new_fitness > current_fitness or random() < np.exp(-(current_fitness[1] - new_fitness[1]) / temperature):
            current_solution = new_solution
            current_fitness = new_fitness

            if current_fitness > best_fitness:
                best_solution = current_solution
                best_fitness = current_fitness
        #Cool gradually the temperature
        temperature = max(temperature * cooling_rate, min_temp)

        history.append(current_fitness[1])

        if temperature <= min_temp:
            break
        ic(iteration, best_fitness, temperature)

    return best_solution, best_fitness, history

## Simulated Annealing algorithm is applied to minimize the total cost of the universe, and the progress is visualized in a plot showing the cost over iterations.

In [ ]:
for universe_size, num_sets, density in zip(UNIVERSE_SIZES, NUM_SETS_LIST, DENSITIES):
    print(f"Solving for Universe Size: {universe_size}, Num Sets: {num_sets}, Density: {density}")
    rng = np.random.Generator(np.random.PCG64([universe_size, num_sets, int(10_000 * density)]))

    SETS = rng.random((num_sets, universe_size)) < density
    for s in range(universe_size):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(num_sets), s] = True
    COSTS = np.pow(SETS.sum(axis=1), 1.1)
    
    best_solution, best_cost, history = simulated_annealing(SETS, COSTS, num_sets, density, rng)
    
    print(f"Best solution cost for Universe Size {universe_size}: {best_cost}")
    print(f"Best solution found: {best_solution}")

    plt.figure(figsize=(14, 8))
    plt.plot(range(len(history)), history, color="blue", label="Fitness over iterations")
    plt.scatter(range(len(history)), history, color="red", marker=".", label="Individual fitness")
    plt.title(f"Simulated Annealing Progress for Universe Size {universe_size}")
    plt.xlabel("Iterations")
    plt.ylabel("Fitness (negative cost)")
    plt.legend()
    plt.grid(True)
    plt.show()